In [1]:
import os
import pandas as pd
import numpy as np
from pprint import pprint
from pathlib import Path
from collections import Counter
import pickle
import random
import argparse
import time
from datetime import datetime

# torch:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.model_selection import train_test_split

from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor
from transformers import BertTokenizer, BertConfig, BertModel,XLMTokenizer, XLMModel
import librosa
from Dementia_challenge_models import Embsize_map
class Arg:
    version = 1
    # data
    epochs: int = 5  # Max Epochs, BERT paper setting [3,4,5]
    max_length: int = 350  # Max Length input size
    report_cycle: int = 30  # Report (Train Metrics) Cycle
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    a_hidden_size = 512 # BERT-base: 768, BERT-large: 1024, BERT paper setting
    t_hidden_size = 768
    aug_hidden_size = Embsize_map['t_hidden_size']['Embedding']
    t_x_hidden_size = a_hidden_size+t_hidden_size+aug_hidden_size
    batch_size: int = 8
            
class BertPooler(nn.Module):
    def __init__(self,hidden_size):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output
    
    
class Model(LightningModule):
    def __init__(self, args,config):
        super().__init__()
        # config:
        
        self.args = args
        self.config = config
        self.batch_size = self.args.batch_size
        
        # meta data:
        self.epochs_index = 0
        self.label_cols = 'dementia_labels'
        self.label_names = ['Control','ProbableAD']
        self.num_labels = 2
        self.t_embed_type = self.config['t_embed']
        self.a_embed_type = self.config['a_embed']
        self.a_hidden = self.args.a_hidden_size
        
        # --> HERE STEP 1 <--
        # ATTRIBUTES TO SAVE BATCH OUTPUTS
        self.test_step_outputs = []   # save outputs in each batch to compute metric overall epoch
        self.test_step_targets = []   # save targets in each batch to compute metric overall epoch
        self.val_step_outputs = []        # save outputs in each batch to compute metric overall epoch
        self.val_step_targets = []        # save targets in each batch to compute metric overall epoch


        if self.t_embed_type == "mbert":
            self.t_hidden = self.args.t_hidden_size
            
            t_pretrained = 'bert-base-multilingual-uncased'
            self.t_tokenizer = BertTokenizer.from_pretrained(t_pretrained)
            self.t_model = BertModel.from_pretrained(t_pretrained)
            
            
        elif self.t_embed_type == "xlm":
            self.t_hidden = self.args.t_hidden_size
            
            t_pretrained = 'xlm-mlm-100-1280'
            self.t_tokenizer = XLMTokenizer.from_pretrained(t_pretrained)
            self.t_model = XLMModel.from_pretrained(t_pretrained)
            self.pooler = BertPooler(self.t_hidden)
            
        self.hidden = int(self.args.t_x_hidden_size)
        
        if self.a_embed_type == "en":
            a_pretrained =  "jonatasgrosman/wav2vec2-large-xlsr-53-english"
            
        elif self.a_embed_type == "gr":
            a_pretrained =  "lighteternal/wav2vec2-large-xlsr-53-greek"

        elif self.a_embed_type == "multi":
            a_pretrained = "voidful/wav2vec2-xlsr-multilingual-56"
            
        elif self.a_embed_type == "wv":
            a_pretrained ='facebook/wav2vec2-base'
            
        self.a_tokenizer = Wav2Vec2FeatureExtractor.from_pretrained(a_pretrained)
        self.a_model = Wav2Vec2Model.from_pretrained(a_pretrained)
        
        
        self.clf1 = nn.Linear(self.hidden, int(self.hidden/2))
        self.clf2 = nn.Linear(int(self.hidden/2), self.num_labels)
        
            
            
    def forward(self, text, audio, Aug):
    # def forward(self, text):
        
        if self.t_embed_type == "mbert":
            t_out = self.t_model(text)[1] 

            
        elif self.t_embed_type == "xlm":
            t_out = self.t_model(text)[0]
            t_out = self.pooler(t_out)
            
            
        a_out = self.a_model(audio)['extract_features']#[2] #last_hidden_state , feature extraction
        a_out = a_out[:, 0, :] 
        
        # print(a_out)
        # print(a_out['extract_features'].shape) # ([8, 437, 512])
        # print(a_out['last_hidden_state'].shape) # ([8, 437, 1024]) => pooling 필요
        
        
        output = torch.cat((t_out,a_out,Aug),axis=1)   
        # output = t_out
        # print(output.shape)
        
        logits = self.clf2(self.clf1(output))
    
        return logits
        

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'])
        scheduler = ExponentialLR(optimizer, gamma=0.5)
        
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def preprocess_dataframe(self):
        
        tg_sr = 16000
        t_col_name = "text" 
        a_col_name = "path"         
        # df = pd.read_json('/mnt/Internal/FedASR/Data/230126_total_asr_data.json')
        df = pd.read_csv(config.file_in)

        df[t_col_name] = df[t_col_name].map(lambda x: self.t_tokenizer.encode(
            str(x),
            padding = 'max_length',
            max_length=self.args.max_length,
            truncation=True,
            ))
        
        audio_root="/mnt/Internal/FedASR/Data/ADReSS-IS2020-data/clips"
        # 원래 길이: 562992, batch 16: 90000, batch 8: 140000
        # max_length=16000, truncation=True 이건 일단 돌려보고 결정 => 뒤쪽, 앞에쪽 뭐보면 좋을 지 그런거 check하면 좋으니까! 
        df[a_col_name] = df[a_col_name].map(lambda x: self.a_tokenizer(
            librosa.load(f"{audio_root}/{x}")[0],
            padding='max_length',
            sampling_rate = tg_sr,
            max_length=100000, 
            truncation=True
            )['input_values'][0])
        
        

        df_train = df[df['ex'] == 'train']
        df_val = df[df['ex'] == 'dev']
        df_test = df[df['ex'] == 'test']


        print(f'# of train:{len(df_train)}, val:{len(df_val)}, test:{len(df_test)}')
        self.t_col_name=t_col_name
        self.a_col_name=a_col_name
        self.df_train=df_train
        self.df_val=df_val
        self.df_test=df_test
    def merge_DataAug2Data(self):
        pname_col_name='ID   '
        similar_col_name='session'
        def AppendID(df_data):
            if pname_col_name not in df_data.columns:
                df_data[pname_col_name]=df_data[similar_col_name]
        AppendID(self.df_train_aug)
        AppendID(self.df_val_aug)
        AppendID(self.df_test_aug)
        # self.df_train = pd.merge(self.df_train, self.df_train_aug, on='ID   ', how='left')
        # self.df_val = pd.merge(self.df_val, self.df_val_aug, on='ID   ', how='left')
        # self.df_test = pd.merge(self.df_test, self.df_test_aug, on='ID   ', how='left')
        self.df_train = pd.merge(self.df_train, self.df_train_aug, on='ID   ', how='left', suffixes=('', '_aug'))
        self.df_val = pd.merge(self.df_val, self.df_val_aug, on='ID   ', how='left', suffixes=('', '_aug'))
        self.df_test = pd.merge(self.df_test, self.df_test_aug, on='ID   ', how='left', suffixes=('', '_aug'))
        
    def preprocess_loaded_summaries(self):
        df_train = pd.read_pickle(f"{config.summary_dir_in}/train.pkl")
        df_val = pd.read_pickle(f"{config.summary_dir_in}/dev.pkl")
        df_test = pd.read_pickle(f"{config.summary_dir_in}/test.pkl")

        Aug_col_name='Embedding'

        def Tokenize(df_data):
            df_data[Aug_col_name] = df_data[Aug_col_name].map(lambda x: self.t_tokenizer.encode(
                str(x),
                padding = 'max_length',
                max_length=self.args.max_length,
                truncation=True,
                ))
            return df_data
        df_train=Tokenize(df_train)
        df_val=Tokenize(df_val)
        df_test=Tokenize(df_test)
        df_test = df_test.reset_index(drop=True)
        self.df_train_aug=df_train
        self.df_val_aug=df_val
        self.df_test_aug=df_test
        self.Aug_col_name=Aug_col_name
    def df2Dataset(self):
        self.train_data = TensorDataset(
            torch.tensor(self.df_train[self.t_col_name].tolist(), dtype=torch.long),
            torch.tensor(self.df_train[self.a_col_name].tolist(), dtype=torch.float),
            torch.tensor(self.df_train[self.Aug_col_name].tolist(), dtype=torch.float),
            torch.tensor(self.df_train[self.label_cols].tolist(), dtype=torch.long),
        )
        
        self.val_data = TensorDataset(
             torch.tensor(self.df_val[self.t_col_name].tolist(), dtype=torch.long),
             torch.tensor(self.df_val[self.a_col_name].tolist(), dtype=torch.float),
             torch.tensor(self.df_val[self.Aug_col_name].tolist(), dtype=torch.float),
            torch.tensor(self.df_val[self.label_cols].tolist(), dtype=torch.long),
        )

        self.test_data = TensorDataset(
             torch.tensor(self.df_test[self.t_col_name].tolist(), dtype=torch.long),
             torch.tensor(self.df_test[self.a_col_name].tolist(), dtype=torch.float),
             torch.tensor(self.df_test[self.Aug_col_name].tolist(), dtype=torch.float),
            torch.tensor(self.df_test[self.label_cols].tolist(), dtype=torch.long),
             torch.tensor(self.df_test.index.tolist(), dtype=torch.long),
        )

    def train_dataloader(self):
        
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )
    
    def val_dataloader(self):

        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )
    
    def test_dataloader(self):

        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )
    
    def training_step(self, batch, batch_idx):
        token, audio, Aug,labels = batch  
        # token,  labels = batch  
        logits = self(token, audio, Aug) 
        # logits = self(token) 
        loss = nn.CrossEntropyLoss()(logits, labels)   
        
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        token, audio, Aug, labels = batch  
        # token, labels = batch  
        logits = self(token, audio, Aug) 
        # logits = self(token) 
        loss = nn.CrossEntropyLoss()(logits, labels)     
        
        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        # --> HERE STEP 2 <--
        self.val_step_outputs.append({
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        })
        # self.val_step_targets.append(y_true)
        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }
        
            
    def test_step(self, batch, batch_idx):
        token, audio, Aug, labels,id_ = batch 
        # token, labels,id_ = batch 
        print('id', id_)
        logits = self(token, audio, Aug) 
        # logits = self(token) 
        
        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        # --> HERE STEP 2 <--
        self.test_step_outputs.append({
            'y_true': y_true,
            'y_pred': y_pred,
        })
        # self.test_step_targets.append(y_true)
        return {
            'y_true': y_true,
            'y_pred': y_pred,
        }
    
    def on_validation_epoch_end(self):
        loss = torch.tensor(0, dtype=torch.float)
        # print("Value= ",self.val_step_outputs)
        # print("type(self.val_step_outputs)=",type(self.val_step_outputs))
        # print("type(self.val_step_outputs[0])=",type(self.val_step_outputs[0]))
        # print("type(self.val_step_outputs[0] loss)=",type(self.val_step_outputs[0]['loss']))
        for i in self.val_step_outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(self.val_step_outputs)
        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in self.val_step_outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
            
        y_pred = np.asanyarray(y_pred)#y_temp_pred y_pred
        y_true = np.asanyarray(y_true)
        
        pred_dict = {}
        pred_dict['y_pred']= y_pred
        pred_dict['y_true']= y_true
        
        val_acc = accuracy_score(y_true=y_true, y_pred=y_pred)
        
        self.log("val_acc", val_acc)
        # print("y_pred= ", y_pred)
        # print('\n\n\n')
        # print("y_true= ", y_true)
        # print('\n\n\n')
        print("-------val_report-------")
        metrics_dict = classification_report(y_true, y_pred,zero_division=1,
                                             target_names = self.label_names, 
                                             output_dict=True)
        df_result = pd.DataFrame(metrics_dict).transpose()
        pprint(df_result)
        

        df_result.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_val.csv')

        pred_df = pd.DataFrame(pred_dict)
        pred_df.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_val_pred.csv')
        self.val_step_outputs.clear()
        # self.val_step_targets.clear()
        return {'loss': _loss}

    def on_test_epoch_end(self):

        y_true = []
        y_pred = []

        for i in self.test_step_outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
            
        y_pred = np.asanyarray(y_pred)#y_temp_pred y_pred
        y_true = np.asanyarray(y_true)
        
        pred_dict = {}
        pred_dict['y_pred']= y_pred
        pred_dict['y_true']= y_true
        
        
        print("-------test_report-------")
        metrics_dict = classification_report(y_true, y_pred,zero_division=1,
                                             target_names = self.label_names, 
                                             output_dict=True)
        df_result = pd.DataFrame(metrics_dict).transpose()
        self.test_step_outputs.clear()
        # self.test_step_targets.clear()
        pprint(df_result)

        df_result.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_test.csv')

        pred_df = pd.DataFrame(pred_dict)
        pred_df.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_test_pred.csv')

    
def main(args,config):
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", config['random_seed'])
    seed_everything( config['random_seed'])
        
    model = Model(args,config) 
    model.preprocess_dataframe()
    model.preprocess_loaded_summaries()
    model.merge_DataAug2Data()
    model.df2Dataset()
    early_stop_callback = EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=True,
        mode='max'
    )
    
    checkpoint_callback = ModelCheckpoint(
        dirpath=f"{SaveRoot}/Model/checkpoints",
        monitor='val_acc',
        auto_insert_metric_name=True,
        verbose=True,
        mode='max', 
        save_top_k=1,
      )    

    print(":: Start Training ::")
    #     
    trainer = Trainer(
        logger=False,
        callbacks=[early_stop_callback,checkpoint_callback],
        enable_checkpointing = True,
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        deterministic=False, # ensure full reproducibility from run to run you need to set seeds for pseudo-random generators,
        # For GPU Setup
        # gpus=[config['gpu']] if torch.cuda.is_available() else None,
        # strategy='ddp_find_unused_parameters_true',
        precision=16 if args.fp16 else 32
    )
    trainer.fit(model)
    trainer.test(model,dataloaders=model.test_dataloader(),ckpt_path="best")
    
if __name__ == '__main__': 

    parser = argparse.ArgumentParser("main.py", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--gpu", type=int, default=1)
    parser.add_argument("--epochs", type=int, default=5)
    parser.add_argument("--lr", type=float, default=2e-5, help="learning rate")
    parser.add_argument("--random_seed", type=int, default=2023) 
    parser.add_argument("--t_embed", type=str, default="mbert") 
    parser.add_argument("--a_embed", type=str, default="en") 
    parser.add_argument("--SaveRoot", type=str, default='/mnt/External/Seagate/FedASR/LLaMa2/dacs') 
    parser.add_argument("--file_in", type=str, default='/home/FedASR/dacs/centralized/saves/results/data2vec-audio-large-960h_total.csv') 
    parser.add_argument("--summary_dir_in", type=str, default='/mnt/External/Seagate/FedASR/LLaMa2/dacs/EmbFeats/Lexical/Embeddings/text_data2vec-audio-large-960h_Phych-anomia') 
    
    config = parser.parse_args(args=[])
    SaveRoot=config.SaveRoot
    
    __file__ = os.path.abspath("__file__")
    script_path, file_extension = os.path.splitext(__file__)

    # 使用os.path模組取得檔案名稱
    script_name = os.path.basename(script_path)

    Output_dir=f"{SaveRoot}/result/{script_name}/"
    os.makedirs(Output_dir, exist_ok=True)

    print(config)
    args = Arg()
    args.epochs=config.epochs
    main(args,config.__dict__)       


"""

python 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed en
python 0207_DM_multi.py --gpu 1 --t_embed xlm --a_embed en

# don
python 0207_DM_multi.py --gpu 0 --t_embed xlm --a_embed gr
python 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed gr

"""

/home/FedASR/.conda/envs/openai/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Seed set to 2023


Namespace(gpu=1, epochs=5, lr=2e-05, random_seed=2023, t_embed='mbert', a_embed='en', SaveRoot='/mnt/External/Seagate/FedASR/LLaMa2/dacs', file_in='/home/FedASR/dacs/centralized/saves/results/data2vec-audio-large-960h_total.csv', summary_dir_in='/mnt/External/Seagate/FedASR/LLaMa2/dacs/EmbFeats/Lexical/Embeddings/text_data2vec-audio-large-960h_Phych-anomia')
Using PyTorch Ver 2.1.1+cu121
Fix Seed: 2023


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# of train:1868, val:206, test:800


/tmp/ipykernel_2823457/1444650004.py:250: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.tensor(self.df_train[self.a_col_name].tolist(), dtype=torch.float),
Trainer will use only 1 of 5 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=5)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-o

...

-------val_report-------
              precision    recall  f1-score     support
Control        0.534653  0.432000  0.477876  125.000000
ProbableAD     0.323810  0.419753  0.365591   81.000000
accuracy       0.427184  0.427184  0.427184    0.427184
macro avg      0.429231  0.425877  0.421734  206.000000
weighted avg   0.451749  0.427184  0.433725  206.000000


Restoring states from the checkpoint path at /mnt/External/Seagate/FedASR/LLaMa2/dacs/Model/checkpoints/epoch=2-step=702.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
Loaded model weights from the checkpoint at /mnt/External/Seagate/FedASR/LLaMa2/dacs/Model/checkpoints/epoch=2-step=702.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

id tensor([0, 1, 2, 3, 4, 5, 6, 7], device='cuda:0')
id tensor([ 8,  9, 10, 11, 12, 13, 14, 15], device='cuda:0')
id tensor([16, 17, 18, 19, 20, 21, 22, 23], device='cuda:0')
id tensor([24, 25, 26, 27, 28, 29, 30, 31], device='cuda:0')
id tensor([32, 33, 34, 35, 36, 37, 38, 39], device='cuda:0')
id tensor([40, 41, 42, 43, 44, 45, 46, 47], device='cuda:0')
id tensor([48, 49, 50, 51, 52, 53, 54, 55], device='cuda:0')
id tensor([56, 57, 58, 59, 60, 61, 62, 63], device='cuda:0')
id tensor([64, 65, 66, 67, 68, 69, 70, 71], device='cuda:0')
id tensor([72, 73, 74, 75, 76, 77, 78, 79], device='cuda:0')
id tensor([80, 81, 82, 83, 84, 85, 86, 87], device='cuda:0')
id tensor([88, 89, 90, 91, 92, 93, 94, 95], device='cuda:0')
id tensor([ 96,  97,  98,  99, 100, 101, 102, 103], device='cuda:0')
id tensor([104, 105, 106, 107, 108, 109, 110, 111], device='cuda:0')
id tensor([112, 113, 114, 115, 116, 117, 118, 119], device='cuda:0')
id tensor([120, 121, 122, 123, 124, 125, 126, 127], device='cuda:0')


'\n\npython 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed en\npython 0207_DM_multi.py --gpu 1 --t_embed xlm --a_embed en\n\n# don\npython 0207_DM_multi.py --gpu 0 --t_embed xlm --a_embed gr\npython 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed gr\n\n'

In [2]:
print(args.a_hidden_size,args.t_hidden_size,args.aug_hidden_size,args.t_x_hidden_size)

512 768 350 1630


In [3]:
args = Arg()
args.epochs=config.epochs
model=Model(args, config.__dict__)
model.preprocess_dataframe()
model.preprocess_loaded_summaries()
model.merge_DataAug2Data()
model.df2Dataset()
self=model


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# of train:1868, val:206, test:800


In [4]:
for batch in model.val_data:
    token, audio, Aug, labels = batch  
    # token, labels = batch  
    logits = self(token, audio, Aug) 
    # logits = self(token) 
    loss = nn.CrossEntropyLoss()(logits, labels)     

    preds = logits.argmax(dim=-1)

    y_true = list(labels.cpu().numpy())
    y_pred = list(preds.cpu().numpy())


IndexError: too many indices for tensor of dimension 1

In [ ]:
print(audio.shape)
print(token.shape)
print(Aug.shape)
model.val_data[0]
model.df_train

torch.Size([100000])
torch.Size([350])
torch.Size([])


,path,text,dementia_labels,pred_str,ID,mmse,ex,path_aug,text_aug,dementia_labels_aug,...,session,role,number,start_time,end_time,Embedding,Psych_Summary,Psych_Prompt,Similarity_Emb,Similarity_IDs
0,"[-0.0026251145, -0.0063604997, -0.009782849, -...","[101, 10103, 12291, 11378, 10127, 21812, 16053...",0,THE STOOL IS TIPPING OT THE LITTLE BOY AND HE'...,S015,29.0,train,S015_PAR_7_26693_32357,INV: AND HERE'S THE PICTURE\nPAR: ALL OF THE A...,0,...,S015,INV+PAR,7,0,56600,"[101, 138, 118, 121, 119, 63547, 62338, 48626,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.017662624105078045, 0.0017859735411924677,...",S015/S154/S020/S033/S071
1,"[-0.03724482, -0.06742435, -0.07992418, -0.093...","[101, 10197, 14650, 10346, 11332, 102, 0, 0, 0...",0,IT MUST BE JUNE,S076,28.0,train,S076_PAR_8_37400_38800,INV: LOOK AT THAT PICTURE\nPAR: MHM\nINV: AND ...,0,...,S076,INV+PAR,8,0,43563,"[101, 138, 118, 121, 119, 75188, 95584, 11444,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.01690500758420462, 0.021220975649458663, 0...",S076/S095/S089/S156/S126
2,"[0.3376349, 0.4526255, 0.3523399, 0.32248497, ...","[101, 151, 11530, 112, 162, 11811, 10197, 2238...",0,I DON'T SEE IT SNOWING,S018,29.0,train,S018_PAR_12_68242_69305,INV: OKAY\nINV: AND THERE'S THE PICTURE\nPAR: ...,0,...,S018,INV+PAR,12,0,71267,"[101, 138, 118, 121, 119, 65945, 92425, 79225,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.018221234206087917, 0.0172382670926329, 0....",S018/S032/S089/S048/S006
3,"[0.0017839464, 0.00059347186, 0.00038148474, 0...","[101, 10103, 20098, 10453, 40811, 10127, 11481...",0,THE COOKIE JAR IS OPEN OF COURSE THE CUPBOARD'...,S062,30.0,train,S062_PAR_2_14910_19143,PAR: WELL THERE'S A KID STEALING COOKIES FROM ...,0,...,S062,INV+PAR,2,0,41000,"[101, 138, 118, 121, 119, 77694, 26622, 42299,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.014507318536576209, 0.01572004712565187, 0...",S062/S001/S118/S083/S126
4,"[-0.0016942845, 0.022142626, 0.02669364, -0.01...","[101, 10103, 11917, 112, 161, 16484, 10323, 10...",0,THE WATER'S RUNNING OVER,S059,30.0,train,S059_PAR_4_14123_15540,INV: ON IN THE PICTURE\nPAR: GIRL IS REACHING ...,0,...,S059,INV+PAR,4,0,58323,"[101, 138, 118, 121, 119, 77694, 11444, 95732,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.017170826914434424, 0.0022816185693933216,...",S059/S103/S033/S071/S020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,"[0.0042455685, 0.0053692805, 0.0051270816, 0.0...","[101, 20098, 10453, 40811, 102, 0, 0, 0, 0, 0,...",1,COOKIE JAR,S128,16.0,train,S128_INV_3_42878_48636,INV: OKAY GOOD\nINV: GOING ON IN THE PICTURE\n...,0,...,S128,INV+PAR,3,0,50925,"[101, 138, 118, 121, 119, 75188, 41282, 82386,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.0186653762774538, 0.00418699146385226, 0.0...",S128/S055/S086/S103/S059
1864,"[-0.3383731, -0.5167495, -0.51931936, -0.54898...","[101, 10110, 10191, 112, 161, 27948, 20098, 11...",1,AND HE'S GETTING COOKIES,S093,25.0,train,S093_PAR_13_60915_66073,INV: WHAT'S GOING ON IN THE PICTURE\nPAR: UH W...,1,...,S093,INV+PAR,13,0,81475,"[101, 138, 118, 121, 119, 69560, 66865, 26622,...",detected problems:\n- Empty speech: PAR's utte...,psycological definition:\n\n - defi...,"[-0.017022879645798548, 0.019089973114330444, ...",S093/S028/S080/S156/S100
1865,"[-0.0017480814, -0.0034497022, -0.004737093, -...","[101, 10572, 112, 161, 15517, 87016, 143, 4683...",1,SHE'S GOT UH A DRESS ON AND AND THERE'S SHE'S ...,S142,14.0,train,S142_INV_0_328_4357,INV: OKAY CAN YOU TELL ME EVERYTHING YOU SEE G...,0,...,S142,INV+PAR,0,328,78752,"[101, 138, 118, 121, 119, 77694, 10995, 80575,...",detected problems:\n- Empty speech: PAR's utte...,psycological defin

In [ ]:
merged_df = pd.merge(self.df_val, self.df_val_aug, on='ID   ', how='left', suffixes=('', '_aug'))

print(model.df_val_aug.columns)
print(model.df_test.columns)
print(len(model.df_test.iloc[0]['Embedding']))
print(len(model.df_test.iloc[0]['Similarity_Emb']))
print(len(model.df_train.iloc[0]['Embedding']))
print(len(model.df_train.iloc[0]['Similarity_Emb']))
# print()
print(len(model.df_val_aug.iloc[0]['Embedding']))
print(len(model.df_val_aug.iloc[0]['Similarity_Emb']))
self.df_val_aug
merged_df
self.df_val
# self.df_test
# self.df_train

In [ ]:
model.preprocess_loaded_summaries()
print(self.df_test.columns)
print(self.df_test_aug.columns)
print(len(self.df_test))
print(len(self.df_test_aug))
merged_df = pd.merge(self.df_test, self.df_test_aug, on='ID   ', how='left')
print(merged_df.columns)



In [ ]:
dataset=self.test_data
# 创建一个 DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# 获取下一个项目
iterator = iter(dataloader)
next_item = next(iterator)

In [ ]:
print(next_item)

In [ ]:
self=model
for batch in model.train_data:
    token, audio, labels = batch 
    # token, labels,id_ = batch 
    print('id', id_)
    logits = self(token, audio) 
    # logits = self(token) 
    
    preds = logits.argmax(dim=-1)

    y_true = list(labels.cpu().numpy())
    y_pred = list(preds.cpu().numpy())

    # --> HERE STEP 2 <--
    # self.test_step_outputs.append({
    #     'y_true': y_true,
    #     'y_pred': y_pred,
    # })

In [ ]:
text, audio= token, audio
print(audio)
if self.t_embed_type == "mbert":
    t_out = self.t_model(text)[1] 